In [ ]:
# import libraries

from scipy.stats import sem as sem
import sys
import os
import glob
import BioSimSpace as BSS

if "/home/anna/Documents/cinnabar" not in sys.path:
    sys.path.insert(1, "/home/anna/Documents/cinnabar")
import cinnabar

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(cinnabar.__file__)

from pipeline import *
from pipeline.utils import *
from pipeline.analysis import *

In [ ]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "tyk2"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"

if os.path.exists(f"{main_dir}/outputs_extracted/results"):
    results_folder = f"{main_dir}/outputs_extracted/results"
elif os.path.exists(f"{main_dir}/outputs/results"):
    results_folder = f"{main_dir}/outputs/results"
# else:
#     raise ValueError(f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline")

output_folder = validate.folder_path(
    f"{main_dir}/analysis/comparing_methods", create=True
)

In [ ]:
engine = "AMBER"
all_analysis_object = analysis_network(
    results_folder,
    exp_file=exp_file,
    net_file=net_file,
    engines=engine,
    output_folder=output_folder,
    analysis_ext=ana_file,
)

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute(cycle_closure=False)

In [ ]:
# can write modified results files

write_modified_results_files(
    f"{results_folder}/final_summary_GROMACS_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv",
    name="switch",
    engine="GROMACS",
)

In [ ]:
print(all_analysis_object.successful_runs(eng=engine))
# all_analysis_object.draw_graph()
all_analysis_object.remove_perturbations(all_analysis_object.failed_runs(eng=engine))
all_analysis_object.remove_ligands(all_analysis_object.disconnected_ligands(eng=engine))
all_analysis_object.compute(cycle_closure=True)
all_analysis_object.draw_graph()

In [ ]:
for eng in all_analysis_object.engines:
    other_results = glob.glob(
        f"{bench_folder}/extracted/{protein}/outputs_extracted/results/final_summary_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    bound_results = glob.glob(
        f"{bench_folder}/extracted/{protein}/outputs_extracted/results/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    free_results = glob.glob(
        f"{bench_folder}/extracted/{protein}/outputs_extracted/results/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    all_analysis_object.compute_other_results(
        other_results,
        name=f"prev_{eng}",
        method=None,
        bound_files=bound_results,
        free_files=free_results,
    )

In [ ]:
eng = "GROMACS"

for name in ["ambersc3", "ambersc", "scamber2"]:
    other_results = glob.glob(
        f"{results_folder}/final_summary_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    bound_results = glob.glob(
        f"{results_folder}/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    free_results = glob.glob(
        f"{results_folder}/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    all_analysis_object.compute_other_results(
        other_results,
        name=f"{name}",
        method=f"{name}",
        use_cinnabar=False,
        bound_files=bound_results,
        free_files=free_results,
    )

In [ ]:
perts, ligs = get_info_network(
    "/backup/42_reruns/tyk2/execution_model/network_combined_reruns_ejm42.dat"
)

for eng in all_analysis_object.engines:
    other_results = glob.glob(
        f"/backup/42_reruns/tyk2/outputs_extracted/results/final_summary_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    all_analysis_object.compute_other_results(other_results, name=f"{eng}_reruns")
    # all_analysis_object.plot_other_results(
    #     name=f"{eng}_reruns", engine=eng, pert_val="pert"
    # )
    all_analysis_object.plot_bar_pert(
        engine=[eng, f"{eng}_reruns"], **{"values": perts}
    )
    res1 = all_analysis_object._stats_object.compute_mue("pert", y=eng)
    res2 = all_analysis_object._stats_object.compute_mue("pert", y=f"{eng}_reruns")
    res3 = all_analysis_object._stats_object.compute_mue(
        "pert", y=eng, x=f"{eng}_reruns"
    )
    print(
        f"og vs exp is {res1[0]}, reruns vs exp is {res2[0]}, and compared to each other theya re {res3[0]}"
    )

all_analysis_object.plot_bar_pert(
    engine=all_analysis_object.engines + all_analysis_object.other_results_names,
    **{"values": perts},
)

compare to the fwf analysis

In [ ]:
# choose an engine
eng = "SOMD"

# first need to add the fwf path
all_analysis_object._add_fwf_path(
    "/home/anna/Documents/freenrgworkflows/networkanalysis"
)

# get the experimental dictionary
exp_dicts = all_analysis_object._get_exp_fwf()
for key in exp_dicts[0]:
    print(f"{key} : {exp_dicts[0][key][0]}, {exp_dicts[0][key][1]}")

# get the network analysis
fwf_dict = all_analysis_object._get_ana_fwf(engine=eng)
for key in fwf_dict:
    print(f"{key} : {fwf_dict[key][0]}, {fwf_dict[key][1]}")

# get fwf stats
fwf_stats = all_analysis_object._get_stats_fwf(engine=eng)

plotting just fwf data per ligand

In [ ]:
dict_y = fwf_dict
dict_exp = exp_dicts[0]

df1 = plotting_engines.match_dicts_to_df(dict_exp, dict_y, "experimental", "fwf")
df1

df1.plot.bar(
    y=["freenrg_fwf", "freenrg_experimental"],
    yerr=df1[["err_fwf", "err_experimental"]].T.values,
    title=f"fwf, experimental, {eng}",
    xlabel="ligands",
    ylabel="dG (kcal/mol)",
)

df1.dropna()
df1.plot.scatter(
    x="freenrg_experimental",
    y="freenrg_fwf",
    xerr="err_experimental",
    yerr="err_fwf",
    title=f"fwf, {eng}",
    xlabel="experimental dG (kcal/mol)",
    ylabel="fwf dG (kcal/mol)",
)

# calculating using the cinnabar stats
f_mae = all_analysis_object._stats_object._compute_stats(
    x=df1["freenrg_experimental"],
    y=df1["freenrg_fwf"],
    xerr=df1["err_experimental"],
    yerr=df1["err_fwf"],
    statistic="MUE",
)
print(f_mae)

# summary of analysis

compare fwf, cinnabar, mbarnet
